## Домашнее задание №1


#### Тема: Языковое моделирование и определение языка.


**Выдана**:   14 сентября 2017

**Дедлайн**:   <font color='red'>9:00 утра 28 сентября 2017</font>

**Среда выполнения**: Jupyter Notebook (Python 3)

#### Правила:

Результат выполнения задания $-$ отчет в формате Jupyter Notebook с кодом и выводами. В ходе выполнения задания требуется реализовать все необходимые алгоритмы, провести эксперименты и ответить на поставленные вопросы. Дополнительные выводы приветствуются. Чем меньше кода и больше комментариев $-$ тем лучше.

Все ячейки должны быть "выполненными", при этом результат должен воспроизвдиться при проверке (на Python 3). Если какой-то код не был запущен или отрабатывает с ошибками, то пункт не засчитывается. Задание, сданное после дедлайна, _не принимается_. Совсем.


Задание выполняется самостоятельно. Вы можете обсуждать идеи, объяснять друг другу материал, но не можете обмениваться частями своего кода. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов, а также предвзято негативное отношение семинаристов в будущем. Если вы нашли в Интернете какой-то код, который собираетесь заимствовать, обязательно укажите это в задании: вполне вероятно, что вы не единственный, кто найдёт и использует эту информацию.

#### Постановка задачи:

В данной лабораторной работе Вам предстоит реализовать n-грамную языковую модель с несколькими видами сглаживания:
- Add-one smoothing
- Stupid backoff
- Interpolation smoothing
- Kneser-Ney smoothing

Вы обучите ее на готовых корпусах, оцените качество и проведете ряд экспериментов. Во второй части задания Вы примените реализованную модель (но с буквенными n-граммами) к задаче распознавания языка. 


# Языковые модели

Цель языкового моделирования заключается в том, чтобы присвоить некоторые вероятности предложениям. Задача состоит в подсчете вероятности $P(W) = P(w_1, \dots, w_n)$ или $P(w_n \mid w_1, \dots, w_{n-1})$. Модель, умеющая вычислять хотя бы одну из этих двух вероятностей, называется **языковой моделью** (LM от Language Model).

Согласно **цепному правилу** (chain rule):

$$P(X_1, \dots, X_n) = P(X_1)P(X_2 \mid X_1)\dots P(X_n \mid X_1, \dots, X_{n-1}).$$ 

Также мы знаем, что

$$
    P(X_n \mid X_1, \dots, X_{n-1}) = \frac{P(X_1, \dots, X_n)}{P(X_1, \dots, X_{n-1})},
$$

следовательно, для того чтобы оценить $P(X_n \mid X_1, \dots, X_{n-1})$ нужно посчитать $P(X_1, \dots, X_n)$ и $P(X_1, \dots, X_{n-1})$. Но эти вероятности будут чрезвычайно малы, если мы возьмем большое $n$, так множество предложений из $n$ слов растет экспоненциально. Для упрощения применим **марковское предположение**: 

$$P(X_n \mid X_1, \dots, X_{n-1}) = P(X_n \mid X_{n - k + 1}, \dots, X_{n-1})$$

для некоторого фиксированного (небольшого) $k$. Это предположение говорит о том, что $X_{n}$ не зависит от $X_{1}, \dots, X_{n - k}$, то есть на следующее слово влияет лишь контекст из предыдущих $k - 1$ слова. Таким образом, мы получаем финальную вероятность:

$$
    P(w_1, \dots, w_n) = \prod_i P(w_i \mid w_{i-k+1}, \dots, w_{i - 1}).
$$

Далее для краткости будем обозначать $w_{i-k}^i := w_{i-k}, \dots, w_{i}$.

## Хранилище n-грам

Для начала выполним вспомогательную работу. Следуйте комментариям, чтобы написать NGramStorage с удобным интерфейсом.

In [1]:
import re
import math
import random
from collections import Counter, defaultdict
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
class NGramStorage:
    """Storage for ngrams' frequencies.
    
    Args:
        sents (list[list[str]]): List of sentences from which ngram
            frequencies are extracted.
        max_n (int): Upper bound of the length of ngrams.
            For instance if max_n = 2, then storage will store
            0, 1, 2-grams.
            
    Attributes:
        max_n (Readonly(int)): Upper bound of the length of ngrams.
    """
        
    def __init__(self, sents=[], max_n=0):
        self.__max_n = max_n
        self.__ngrams = {i: Counter() for i in range(self.__max_n + 1)}
        # self._ngrams[K] should have the following interface:
        # self._ngrams[K][(w_1, ..., w_K)] = number of times w_1, ..., w_K occured in words
        # self._ngrams[0][()] = number of all words
        
        ### YOUR CODE HERE
        for sent in sents:
            tagged_sentence = ['<s>'] + sent + ['</s>']
            for l in range(1, self.__max_n + 1):
                self.__ngrams[l].update([tuple(tagged_sentence[i:i+l]) for i in range(len(tagged_sentence) - l + 1)])
        self.__ngrams[0][()] = sum(self.__ngrams[1].values())
        ### END YOUR CODE
        
    def add_unk_token(self):
        """Add UNK token to 1-grams."""
        # In order to avoid zero probabilites 
        if self.__max_n == 0 or 'UNK' in self.__ngrams[1]:
            return
        self.__ngrams[0][()] += 1
        self.__ngrams[1][('UNK', )] = 1
        
    @property
    def max_n(self):
        """Get max_n"""
        return self.__max_n
        
    def __getitem__(self, k):
        """Get dictionary of k-gram frequencies.
        
        Args:
            k (int): length of returning ngrams' frequencies.
            
        Returns:
            Dictionary (in fact Counter) of k-gram frequencies.
        """
        # Cheking the input
        if not isinstance(k, int):
            raise TypeError('k (length of ngrams) must be an integer!')
        if k > self.__max_n:
            raise ValueError('k (length of ngrams) must be less or equal to the maximal length!')
        return self.__ngrams[k]
    
    def __call__(self, ngram):
        """Return frequency of a given ngram.
        
        Args:
            ngram (tuple): ngram for which frequency should be computed.
            
        Returns:
            Frequency (int) of a given ngram.
        """
        # Cheking the input
        if not isinstance(ngram, tuple):
            raise TypeError('ngram must be a tuple!')
        if len(ngram) > self.__max_n:
            raise ValueError('length of ngram must be less or equal to the maximal length!')
        if len(ngram) == 1 and ngram not in self.__ngrams[1]:
            return self.__ngrams[1][('UNK', )]
        return self.__ngrams[len(ngram)][ngram]

## Оценка качества

Скачайте brown корпус, обучите модель и протестируйте на нескольких примерах последовательностей.

In [3]:
import nltk
# Uncomment next row and download brown corpus
# nltk.download()
from nltk.corpus import brown

In [4]:
all_sents = list(brown.sents())
random.shuffle(all_sents)
print('Number of all sentences = {}'.format(len(all_sents)))
train_sents = all_sents[:int(0.8 * len(all_sents))]
test_sents = all_sents[int(0.8 * len(all_sents)):]
print('Number of train sentences = {}'.format(len(train_sents)))
print('Number of test sentences = {}'.format(len(test_sents)))

Number of all sentences = 57340
Number of train sentences = 45872
Number of test sentences = 11468


In [5]:
# Create storage of 0, 1, 2, 3-grams
storage = NGramStorage(train_sents, 3)

In [6]:
# It's time to test your code
print(storage(('to', 'be')))
print(storage(('or',)))
print(storage(('not', 'to', 'be')))
print(storage(('somethingweird',)))
print(storage(()))

1379
3323
27
0
1021818


Для численного измерения качества языковой модели определим **перплексию**:

$$
    {\mathbb{P}}(w_1, \dots, w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_i P(w_i \mid w_{i - k}, \dots, w_{i - 1})\right)^{-\frac1N},
$$

Вижно, что минимизация перплексии эквивалентна максимизации правдоподобия модели.

Реализуйте функцию по подсчету перплексии. Обратите внимание, что перплексия по корпусу равна произведению вероятностей **всех** предложений в степени $-\frac1N$, где $N -$ суммарная длина всех предложений.

In [7]:
def perplexity(estimator, sents):
    '''Estimate perplexity of the sequence of words using prob_estimator.'''
    ### YOUR CODE HERE
    # Avoid log(0) by replacing zero by 10 ** (-50).
    N = np.sum([len(sent) + 1 for sent in sents]) # adding 1 because of </s> token
    perp = np.exp(-1. / N * np.sum([np.log(np.max([estimator.prob(sentence), 10 ** (-50)])) for sentence in sents]))
    ### END YOUR CODE
    return perp

## Оценка вероятностей n-грам

Первый и простейший способ оценки вероятностей N-грам следующий:

$$
    \hat P_{S}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N)}{c(w_1^{N-1})}.
$$

где $c(w_1^N)$ — это число последовательностей $w_1, \dots, w_N$ в корпусе, $S$ символизирует Straightforward. 

In [8]:
class StraightforwardProbabilityEstimator:
    """Class for simplest probability estimations of type P(word | context).
    
    P(word | context) = c(context + word) / c(context), where
    c(sequence) - number of occurances of the sequence in the corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
    """
    
    def __init__(self, storage):
        self.__storage = storage
        # Adding UNK token to avoid zero probabilities
        self.__storage.add_unk_token()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if self.__storage.max_n == 1:
            return ()
        if len(context) + 1 > self.__storage.max_n:
            return context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        # Avoiding 0 / 0.
        if context_counts == 0:
            return 0.
        return 1. * phrase_counts / context_counts
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.

            Args:
                sentence (list[str]): Sentence for probability estimation.

            Returns:
                Probability (float) P(sentence).
        """
        tagged_sent = ['<s>'] + sent + ['</s>']
        prob = 1.
        for i in range(1, len(tagged_sent)):
            prob *= self(tagged_sent[i], tuple(tagged_sent[:i]))
        return prob

In [9]:
# Initialize estimator
simple_estimator = StraightforwardProbabilityEstimator(storage)

# Estimating perplexity
print('Simple estimator perplexity = {}'.format(perplexity(simple_estimator, test_sents)))
print(simple_estimator.prob('To be'.split()))
print(simple_estimator.prob('To be or not to be'.split()))

Simple estimator perplexity = 204.59333501530597
0.0
0.0


Посчитаем перплексию униграмной модели.

In [10]:
uni_storage = NGramStorage(train_sents, 1)
uni_simple_estimator = StraightforwardProbabilityEstimator(uni_storage)
print('Simple estimator perplexity = {}'.format(perplexity(uni_simple_estimator, test_sents)))

Simple estimator perplexity = 93.36516792484356


Ответьте на следующие вопросы (внутри ipython ноутбука):

**Q:** Какие выводы можно сделать? Почему $P(\text{To be or not to be}) = 0$, хотя мы и добавили UNK токен?  
**A:** Так как в тренировочном корпусе не встретились определенные последовательности слов и их количество равно 0.

**Q:** Почему перплексия униграмной модели меньше, чем триграмной?  
**A:** Так как чем длиннее контекст, тем больше шанс того, что фраза не встретилась в тренировочном корпусе, тогда его вероятность будет равна 0 вместо какой-то адекватной величины.


## Add-one smoothing

Простейший вид сглаживания — **сглаживание Лапласа**. Чтобы избавиться от нулевых вероятностей $P(w_{N} \mid w_1^{N - 1})$, будем использовать формулу:

$$
    \hat P_{AOS}(w_{N} \mid w_1^{N - 1}) = \frac{c(w_1^N) + \delta}{c(w_1^{N-1}) + \delta V},
$$

где $V$ — это размер словаря, а $\delta$ — некоторая фиксированная константа.

Реализуйте класс, осуществляющий сглаживание Лапласа. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [11]:
class LaplaceProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = (c(context + word) + delta) / (c(context) + delta * V), where
    c(sequence) - number of occurances of the sequence in the corpus,
    delta - some constant,
    V - number of different words in corpus.
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): Smoothing parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            return context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('context must be a tuple!')
            
        ### YOUR CODE HERE
        context = self.cut_context(context)
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        prob = 1. * (phrase_counts + self.__delta) / (context_counts + len(self.__storage[1]) * self.__delta)
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.

            Args:
                sentence (list[str]): Sentence for probability estimation.

            Returns:
                Probability (float) P(sentence).
        """
        tagged_sent = ['<s>'] + sent + ['</s>']
        prob = 1.
        for i in range(1, len(tagged_sent)):
            prob *= self(tagged_sent[i], tuple(tagged_sent[:i]))
        return prob

Подберите наилучший параметр $\delta$ для данного корпуса.

In [12]:
# Try to find out best delta parameter. We will not provide you any strater code.
### YOUR CODE HERE
candidates = np.logspace(-10, 2, 13)
candidate_perplexities = np.array(
    [perplexity(LaplaceProbabilityEstimator(storage, delta), train_sents) for delta in candidates] # let me be honest and use train set
)
index_of_best = np.argmin(candidate_perplexities)
best_delta = candidates[index_of_best]
### END YOUR CODE

# Initialize estimator
laplace_estimator = LaplaceProbabilityEstimator(storage, best_delta)

# Let's make some estimations
print('Laplace estimator perplexity = {}'.format(perplexity(laplace_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))

Laplace estimator perplexity = 149.7378501026779
1.9983132733e-15


## Stupid backoff

Идея **простого отката** довольно понятна. Если у нас есть достаточно информцаии для подсчета вероятности $k$-грам, то будем использовать $k$-грамы. Иначе будем использовать вероятности $(k-1)$-грам с некоторым множителем, например, $0.4$, и так далее. К сожалению, в данном случае мы получим не вероятностное распределение, но в большинстве задач это не имеет принципиального значения. Если это все же важно, то необходимо подобрать множитель соответствующим образом.

Реализуйте класс, симулирующий сглаживание простым откатом. Он должен иметь аналогичный интерфейс, как и StraightforwardProbabilityEstimator.

In [13]:
class StupidBackoffProbabilityEstimator:
    """Class for stupid backoff probability estimations.
    
    P(word | context) =
        P'(word | context),                  if  P'(word | context) > 0;
        P'(word | context[1:]) * multiplier, if  P'(word | context) == 0
                                             and P'(word | context[1:]) > 0;
        ...
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        multiplier (float): Multiplier which is used for probability estimations.
    """
    
    def __init__(self, base_estimator, multiplier=0.1):
        self.__base_estimator = base_estimator
        self.__mult = multiplier
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        prob = self.__recursive_inner_call(word, context)
        ### END YOUR CODE
        
        return prob
    
    def __recursive_inner_call(self, word, context):
        if not context:
            return self.__base_estimator(word, context)
        return self.__base_estimator(word, context) or self.__recursive_inner_call(word, context[1:]) * self.__mult
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.

            Args:
                sentence (list[str]): Sentence for probability estimation.

            Returns:
                Probability (float) P(sentence).
        """
        tagged_sent = ['<s>'] + sent + ['</s>']
        prob = 1.
        for i in range(1, len(tagged_sent)):
            prob *= self(tagged_sent[i], tuple(tagged_sent[:i]))
        return prob

In [14]:
# Initialize estimator
sbackoff_estimator = StupidBackoffProbabilityEstimator(simple_estimator, .4)

# Let's make some estimations
print('Stupid backoff estimator perplexity = {}'.format(perplexity(sbackoff_estimator, test_sents)))
print(laplace_estimator.prob('To be'.split()))

Stupid backoff estimator perplexity = 102.8872441974894
1.9983132733e-15


Ответьте на следующие вопросы (внутри ipython ноутбука):

**Q:** Почему бессмысленно измерять перплексию в случае **Stupid backoff**?  
**A:** Так как *Stupid backoff* не порождает вероятностное распределение, то и отдаваемые значения нельзя перемножать как вероятности.


## Interpolation smoothing

В данном случае идея сглаживания посредством **интерполяции** также крайне проста. Пусть у нас есть $N$-грамная модель. Заведем вектор $\bar\lambda = (\lambda_1, \dots, \lambda_N)$, такой, что $\sum_i\lambda_i = 1$ и $\lambda_i \geq 0$. Тогда

$$
    \hat P_{IS}(w_{N} \mid w_1^{N-1}) = \sum_{i=1}^N \lambda_i \hat P_{S}(w_N \mid w_{N-i+1}^{N-1}).
$$

Придумайте, как обойтись одним вектором $\bar\lambda$, т.е. пользоваться им как в случае контекста длины $N$, так и при контексте меньшей длины (например, в начале предложения). Если мы просто обрубим сумму, то у нас уже не будет вероятностное распределение, что, конечно же, плохо.

In [15]:
class InterpolationProbabilityEstimator:
    """Class for interpolation probability estimations.
    
    P(word | context) =
        lambda_N * P'(word | context) +
        lambda_{N-1} * P'(word | context[1:]) +
        ... +
        lambda_1 * P'(word)
    P'(word | context) - probability of a word provided context of a base estimator.
    
    Args:
        base_estimator(BaseProbabilityEstimator): Object of BaseProbabilityEstimator
            or some other class which can estimate conditional probabilities.
        lambdas (np.array[float]): Lambdas which are used for probability estimations.
    """
    
    def __init__(self, base_estimator, lambdas):
        self.lambdas = lambdas
        self.__base_estimator = base_estimator
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        
        ### YOUR CODE HERE
        length = min(len(self.lambdas), len(context) + 1)
        skip = len(context) - length + 1
        true_context = context[skip:]
        estimations = np.array([self.__base_estimator(word, true_context[i:]) for i in range(length)])
        weights = np.array(self.lambdas[-length:])
        weights[0] += np.sum(self.lambdas[:-length])
        prob = np.sum(estimations * weights[::-1])
        ### END YOUR CODE
        
        return prob
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.

            Args:
                sentence (list[str]): Sentence for probability estimation.

            Returns:
                Probability (float) P(sentence).
        """
        tagged_sent = ['<s>'] + sent + ['</s>']
        prob = 1.
        for i in range(1, len(tagged_sent)):
            prob *= self(tagged_sent[i], tuple(tagged_sent[:i]))
        return prob

In [16]:
import numpy as np

# Initialize estimator
interpol_estimator = InterpolationProbabilityEstimator(simple_estimator, np.array([0.2, 0.2, 0.6]))

# Let's make some estimations
print('Interpolation estimator perplexity = {}'.format(perplexity(interpol_estimator, test_sents)))
print(interpol_estimator.prob('To be'.split()))

Interpolation estimator perplexity = 72.72380311908718
1.09030841082e-06


Обучить значения параметров $\lambda$ можно с помощью EM-алгоритма, но мы не будем этого здесь делать.

## Kneser-Ney smoothing

Идея данного сглаживания заключается в том, что словам, которые участвуют в большом количестве контекстов, присваиваются большие вероятности, а те, которые используются в паре-тройке контекстов, получают маленькие вероятности. Формулы приведены на слайде 37 лекции.
Реализуйте данный подход.

In [17]:
class KneserNeyProbabilityEstimator:
    """Class for probability estimations of type P(word | context).
    
    P(word | context) = ...
    
    Args:
        storage(NGramStorage): Object of NGramStorage class which will
            be used to extract frequencies of ngrams.
        delta(float): KneserNey parameter.
    """
    
    def __init__(self, storage, delta=1.):
        self.__storage = storage
        self.__delta = delta
        self.__N = defaultdict(lambda: defaultdict(set))
        self.__precalc_N()
        
    def cut_context(self, context):
        """Cut context if it is too large.
        
        Args:
            context (tuple[str]): Some sequence of words.
        
        Returns:
            Cutted context (tuple[str]) up to the length of max_n.
        """
        if len(context) + 1 > self.__storage.max_n:
            context = context[-self.__storage.max_n + 1:]
        return context
        
    def __call__(self, word, context):
        """Estimate conditional probability P(word | context).
        
        Args:
            word (str): Current word.
            context (tuple[str]): Context of a word.
            
        Returns:
            Conditional probability (float) P(word | context).
        """
        # Cheking the input
        if not isinstance(word, str):
            raise TypeError('word must be a string!')
        if not isinstance(context, tuple):
            raise TypeError('word must be a string!')
        # If context is too large, let's cut it.
        context = self.cut_context(context)
        
        
        
        ### YOUR CODE HERE
        prob = self.__recursive_inner_call(word, context)
        ### END YOUR CODE
        
        return prob
    
    def __recursive_inner_call(self, word, context):
        if not context:
            denominator = self.__N[1][()]
            if not denominator:
                return 0
            return 1. * len(self.__N[0][word]) / denominator
        phrase_counts = self.__storage(context + (word, ))
        context_counts = self.__storage(context)
        if not context_counts:
            return self.__recursive_inner_call(word, context[1:])
        return 1. * max(phrase_counts - self.__delta, 0) / context_counts + \
                1. * self.__delta / context_counts * len(self.__N[len(context) + 1][context]) \
                * self.__recursive_inner_call(word, context[1:])
            
            
    def __precalc_N(self):
        for l, seqs in self.__storage._NGramStorage__ngrams.items():
            if l > 1:
                for seq in seqs.keys():
                    self.__N[l][seq[:-1]].add(seq[-1])
        for pre, after in self.__storage[2]:
            self.__N[0][after].add(pre)
        self.__N[1][()] = sum(map(len, self.__N[2].values()))
    
    def prob(self, sent):
        """Estimate probability of a sentence using Markov rule.

            Args:
                sentence (list[str]): Sentence for probability estimation.

            Returns:
                Probability (float) P(sentence).
        """
        tagged_sent = ['<s>'] + sent + ['</s>']
        prob = 1.
        for i in range(1, len(tagged_sent)):
            prob *= self(tagged_sent[i], tuple(tagged_sent[:i]))
        return prob


In [18]:
# Initialize estimator
kn_estimator = KneserNeyProbabilityEstimator(storage, 1.)

# Estimating perplexity
print('Kneser-Ney estimator perplexity = {}'.format(perplexity(kn_estimator, test_sents)))
print(kn_estimator.prob('To be'.split()))
print(kn_estimator.prob('To be or not to be'.split()))

Kneser-Ney estimator perplexity = 78.52644521740432
1.6170939178025017e-07
8.842635008837775e-15


## Определение языка документа

**Постановка задачи:**  
Одна из задач, которая может быть решена при помощи языковых моделей $-$ **определение языка документа**. Реализуйте два классификатора для определения языка документа:
1. Наивный классификатор, который будет учитывать частоты символов и выбирать язык текста по признаку: распределение частот символов "наиболее похоже" на распределение частот символов в выбранном языке.
2. Классификатор на основе языковых моделей. Сами придумайте, как он должен работать.  
_Подсказка_: лучше считать n-грамы не по словам, а по символам.

---

**Как представлены данные:**  
Во всех текстовых файлах на каждой строчке записано отдельное предложение.
1. В папке _data_ находятся две папки: _full_ и _plain_. В _full_ находятся тексты в той форме, что они были взяты из сети, в _plain_ находятся те же самые тексты, но с них сначала была снята диакритика, а затем русский и греческий тексты были транслитерованы в английский.
2. В каждой из папок _full_ и _plain_ находятся папки _train_ и _test_.
3. В _train_ находятся файлы с текстами с говорящими именами, например, _ru.txt_, _en.txt_.
4. В _test_ находятся файлы _1.txt_, _2.txt_, $\dots$ в которых хранятся тексты, язык которых нужно определить. В этой же папке находится файл _ans.csv_, в котором вы можете найти правильные ответы и проверить, насколько хорошо сработали Ваши алгоритмы.

---

**Что нужно сделать:**  
Напишите два своих классификатора (которые описаны в постановке задачи) и получите максимально возможное accuracy на test-сете. Разрешается использовать только _train_ для обучения.

---

**В данном задании мы не предоставляем стартового кода!**

Для начала напишем наивный классификатор. Принимает на вход словарь с ключами-метками языков и текстом на этом языке.
В качестве оценки близости будет использовано что-то вроде среднеквадратического отклонения частот. 

In [19]:
class NaiveLanguageDetector:
    
    def __init__(self):
        self.__freq_counter = {}
        
    def train(self, texts):
        for lang, text in texts.items():
            self.__freq_counter[lang] = defaultdict(int, {letter: count / len(text) for letter, count in Counter(text).items()})
            
    def predict(self, text):
        frequencies = defaultdict(int, {letter: count / len(text) for letter, count in Counter(text).items()})
        lang_index = np.argmin([self.__get_square_loss(frequencies, freq) for freq in self.__freq_counter.values()])
        return list(self.__freq_counter.keys())[lang_index]
        
    @staticmethod
    def __get_square_loss(lfreq, rfreq):
        symbols = set(list(lfreq.keys()) + list(rfreq.keys()))
        return sum([(lfreq[key] - rfreq[key])**2 for key in symbols])

Считываем данные.

In [20]:
import csv

def read_answers(path):
    answers = {}
    with open(path, 'r') as plain_answers:
        reader = csv.reader(plain_answers)
        answers.update({number: lang for number, lang in reader})
    return answers

In [21]:
from os import listdir, path

def read_texts(texts_path):
    texts = {}
    for file in listdir(texts_path):
        if file.endswith('txt'):
            with open(path.join(texts_path, file), 'r') as text:
                lang = file.split('.txt')[0]
                texts[lang] = text.read()
    return texts

Тестируем на двух датасетах. Как видим, на "плоском" была допущена одна ошибка (есть инсайдерская информация, что перепутались английский с немецким, но я не включил это в вывод ноутбука в финальной версии). На "полном" ни одной.

In [22]:
def test_naive(data_path):
    train_texts = read_texts(path.join(data_path, 'train'))
    test_texts = read_texts(path.join(data_path, 'test'))
    answers = read_answers(path.join(data_path, 'test/ans.csv'))
    classifier = NaiveLanguageDetector()
    classifier.train(train_texts)
    predictions = {text_name: classifier.predict(text) for text_name, text in test_texts.items()}
    mistakes = 0
    for key, val in answers.items():
        if answers[key] != predictions[key]:
            mistakes += 1
    return 1. * (len(answers) - mistakes) / len(answers)

print(test_naive('language_detection/plain'))
print(test_naive('language_detection/full'))

0.9958333333333333
1.0


Перейдем к классификатору на основе наших языковых моделей. Как завещал автор ноутбука строить будем побуквенно, н-грама максимальной длины 3 (потому что сейчас очень поздно, а на больших значениях оно будет долго высчитывать). Различные эвристики оценки степени принадлежности текста к языку приходили мне в голову, но вдохновленный материалом первой части домашнего задания я решил остановиться на выборе минимальной перплексии подсчитанной для каждой модели построенной на основе каждого текста. Понятно, что чем более похож будет текст неизвестного языка на некий текст известного языка, то тем больше там будет высоких вероятностей для предложений, поэтому перплексия на текстах имеет смысл. Честно говоря, думал, что придется менять метод, но в итоге...

In [32]:
class NGramBasedLanguageDetector:
    
    def __init__(self):
        self.__estimators = {}
        
    def train(self, texts):
        for lang, text in texts.items():
            print('training ', lang)
            ok_text = [list(sent) for sent in re.split('[.?!]', text)]
            storage = NGramStorage(ok_text, 3)
            self.__estimators[lang] = KneserNeyProbabilityEstimator(storage)
            print('trained ', lang)
            
    def predict(self, text):
        ok_text = [list(sent) for sent in re.split('[.?!]', text)]
        perplexities = [perplexity(estimator, ok_text) for estimator in self.__estimators.values()]
        index = np.argmin(perplexities)
        return list(self.__estimators.keys())[index]

In [33]:
def test_ngram(data_path):
    train_texts = read_texts(path.join(data_path, 'train'))
    test_texts = read_texts(path.join(data_path, 'test'))
    answers = read_answers(path.join(data_path, 'test/ans.csv'))
    classifier = NGramBasedLanguageDetector()
    classifier.train(train_texts)
    predictions = {text_name: classifier.predict(text) for text_name, text in test_texts.items()}
    mistakes = 0
    for key, val in answers.items():
        if answers[key] != predictions[key]:
            mistakes += 1
    return 1. * (len(answers) - mistakes) / len(answers)

# print(test_ngram('language_detection/plain'))
print(test_ngram('language_detection/full'))

training  ca
trained  ca
training  de
trained  de
training  el
trained  el
training  en
trained  en
training  eo
trained  eo
training  es
trained  es
training  fi
trained  fi
training  fr
trained  fr
training  hu
trained  hu
training  it
trained  it
training  nl
trained  nl
training  no
trained  no
training  pl
trained  pl
training  pt
trained  pt
training  ru
trained  ru
training  sv
trained  sv
0.9958333333333333


Прошу простить мне мой высокотехнологичный прогрессбар. 

Видим, что была допущена лишь одна ошибка при классификации, поэтому результатом я удовлетворен более чем. Если поиграться с длиной н-грамы, то, думаю, на данном датасете без труда можно было бы добиться идеальной точности.